# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import json
import numpy as np
import psycopg2
from psycopg2 import Error
import pandas as pd
from sql_queries import *

In [2]:
dsn = "host=127.0.0.1 dbname=sparkifydb user=student password=student"
def conncur():
    conn = psycopg2.connect(dsn)
    cur = conn.cursor()
    return conn, cur


def conncurclose():
    try:
        cur.close()
        conn.close()
    except Error as e:
        print("Error closing cursor and connection:", e)

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

def my_get_files(path, suffix='.json'):
    """
    Find all absolute paths of files ending in suffix, with root = path.
    
    suffix may be a tuple of strings
    """
    all_files = []
    for root, dirs, files in os.walk(path):
        for f in files:
            if f.endswith(suffix):
                all_files.append(os.path.abspath(f))
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
filepath = song_files[0]

In [6]:
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982


In [7]:
df.dtypes

num_songs             int64
artist_id            object
artist_latitude     float64
artist_longitude    float64
artist_location      object
artist_name          object
song_id              object
title                object
duration            float64
year                  int64
dtype: object

In [8]:
song0 = json.load(open(filepath))
print(song0)

{'num_songs': 1, 'artist_id': 'AR7G5I41187FB4CE6C', 'artist_latitude': None, 'artist_longitude': None, 'artist_location': 'London, England', 'artist_name': 'Adam Ant', 'song_id': 'SONHOTT12A8C13493C', 'title': 'Something Girls', 'duration': 233.40363, 'year': 1982}


## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_cols = ['artist_id', 'artist_name',
               'artist_location', 'artist_latitude',
               'artist_longitude']
artist_data = df.loc[:, artist_cols].values[0].tolist()
artist_data

['AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
conn, cur = conncur()
try:
    cur.execute(artist_table_insert, artist_data)
    conn.commit()
finally:
    conncurclose()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [11]:
song_cols = ['song_id', 'title', 'artist_id', 'year', 'duration']
song_data = df.loc[:, song_cols].values[0].tolist()
song_data

['SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363]

In [12]:
for x in song_data:
    print(type(x))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'int'>
<class 'float'>


In [13]:
ser = pd.Series(list(np.array([1, 2, 3])) + ['abc'])
ser_list = ser.tolist()
ser_list2 = ser.values.tolist()

for x in [ser, ser_list, ser_list2]:
    print(type(x[0]), type(x[3]))

<class 'numpy.int64'> <class 'str'>
<class 'numpy.int64'> <class 'str'>
<class 'numpy.int64'> <class 'str'>


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [14]:
conn, cur = conncur()
try:
    cur.execute(song_table_insert, song_data)
    conn.commit()
except Error as e:
    print("Error:", e)
finally:
    conncurclose()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [15]:
log_files = get_files('data/log_data')

In [16]:
filepath = log_files[0]

In [17]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   artist         73 non-null     object 
 1   auth           95 non-null     object 
 2   firstName      92 non-null     object 
 3   gender         92 non-null     object 
 4   itemInSession  95 non-null     int64  
 5   lastName       92 non-null     object 
 6   length         73 non-null     float64
 7   level          95 non-null     object 
 8   location       92 non-null     object 
 9   method         95 non-null     object 
 10  page           95 non-null     object 
 11  registration   92 non-null     float64
 12  sessionId      95 non-null     int64  
 13  song           73 non-null     object 
 14  status         95 non-null     int64  
 15  ts             95 non-null     int64  
 16  userAgent      92 non-null     object 
 17  userId         95 non-null     object 
dtypes: float64(2

In [19]:
filt = df.loc[:, 'page'] == 'NextSong'
df_next_song = df[filt]
df_next_song.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


In [20]:
t = df_next_song.loc[:, 'ts']
t = pd.to_datetime(t, unit='ms')
t.head()

0   2018-11-11 02:33:56.796
1   2018-11-11 02:36:10.796
2   2018-11-11 02:40:34.796
4   2018-11-11 04:36:13.796
5   2018-11-11 04:36:46.796
Name: ts, dtype: datetime64[ns]

In [21]:
t[0].day

11

In [22]:
t[0].weekday()    

6

In [23]:
def ts_to_time_data(x):
    weekday = x.weekday() not in [5, 6]
    return (x, x.hour, x.day, x.week, x.month, x.year, weekday)

print(ts_to_time_data(t[0]))

(Timestamp('2018-11-11 02:33:56.796000'), 2, 11, 45, 11, 2018, False)


In [24]:
time_data = ()
column_labels = ()

In [25]:
#time_df = 
#time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [26]:
"""
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()
"""

'\nfor i, row in time_df.iterrows():\n    cur.execute(time_table_insert, list(row))\n    conn.commit()\n'

In [27]:
conn, cur = conncur()
try:
    for i, x in t.iteritems():
        tup = ts_to_time_data(x)
        cur.execute(time_table_insert, tup)
        conn.commit()
except Error as e:
    print('Error inserting:', e)
finally:
    conncurclose()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [28]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [29]:
user_cols = ['userId', 'firstName', 'lastName', 'gender', 'level']
user_df = df.loc[:, user_cols]
user_df.head()

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,69,Anabelle,Simpson,F,free
2,69,Anabelle,Simpson,F,free
3,32,Lily,Burns,F,free
4,32,Lily,Burns,F,free


In [30]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     95 non-null     object
 1   firstName  92 non-null     object
 2   lastName   92 non-null     object
 3   gender     92 non-null     object
 4   level      95 non-null     object
dtypes: object(5)
memory usage: 3.8+ KB


In [31]:
filt = user_df.loc[:, 'userId'] == ''
user_df_drop = user_df[~filt]
user_df_drop.head()

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,69,Anabelle,Simpson,F,free
2,69,Anabelle,Simpson,F,free
3,32,Lily,Burns,F,free
4,32,Lily,Burns,F,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [32]:
user_table_insert2 = (user_table_insert.rstrip().rstrip(';')
                      + " ON CONFLICT DO NOTHING;")

In [33]:
conn, cur = conncur()
try:
    for i, row in user_df_drop.iterrows():
        cur.execute(user_table_insert, row)
        conn.commit()
except Error as e:
    print("Error", e)
finally:
    conncurclose()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [34]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [35]:
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69


Columns for `songplays`:

| column | origin |
| --- | --- |
| songplay_id | serial |
| start_time | 'ts' |
| user_id | 'userId' |
|level | 'level' |
|song_id | song_query|
|artist_id|song_query|
|session_id| 'sessionId'|
|location | 'location'|
|user_agent|'userAgent'|

In [36]:
songplay_cols = ['userId', 'level',
                 'sessionId', 'location', 'userAgent']

def songplay_format(row, songid, artistid):
    ts = pd.to_datetime(row.ts, unit='ms')
    temp = row[songplay_cols].tolist()
    return [ts] + temp[:2] + [songid, artistid] + temp[2:]

test = songplay_format(df.loc[0, :], 'abc123', 'def456')
for item in test:
    print(item)
print('\n', df.loc[0, :])

2018-11-11 02:33:56.796000
69
free
abc123
def456
455
Philadelphia-Camden-Wilmington, PA-NJ-DE-MD
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"

 artist                                                    Frumpies
auth                                                     Logged In
firstName                                                 Anabelle
gender                                                           F
itemInSession                                                    0
lastName                                                   Simpson
length                                                     134.478
level                                                         free
location               Philadelphia-Camden-Wilmington, PA-NJ-DE-MD
method                                                         PUT
page                                                      NextSong
registration                             

In [37]:
conn, cur = conncur()
try:
    for index, row in df_next_song.iterrows():

        # get songid and artistid from song and artist tables
        cur.execute(song_select, (row.song, row.artist, row.length))
        results = cur.fetchone()
    
        if results:
            songid, artistid = results
        else:
            songid, artistid = None, None

        # insert songplay record
        songplay_data = tuple(songplay_format(row, songid, artistid))
        cur.execute(songplay_table_insert, songplay_data)
        conn.commit()
finally:
    conncurclose()

Run `test.ipynb` to see if you've successfully added records to this table.

In [38]:
df_next_song.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73 entries, 0 to 94
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   artist         73 non-null     object 
 1   auth           73 non-null     object 
 2   firstName      73 non-null     object 
 3   gender         73 non-null     object 
 4   itemInSession  73 non-null     int64  
 5   lastName       73 non-null     object 
 6   length         73 non-null     float64
 7   level          73 non-null     object 
 8   location       73 non-null     object 
 9   method         73 non-null     object 
 10  page           73 non-null     object 
 11  registration   73 non-null     float64
 12  sessionId      73 non-null     int64  
 13  song           73 non-null     object 
 14  status         73 non-null     int64  
 15  ts             73 non-null     int64  
 16  userAgent      73 non-null     object 
 17  userId         73 non-null     object 
dtypes: float64(2

# Close Connection to Sparkify Database

In [39]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [40]:
conn, cur = conncur()
try:
    cur.execute(song_select)
except Error as e:
    print("Error:", e)

for i, row in enumerate(cur.fetchall()):
    if i < 20:
        print(row)
conncurclose()

Error: syntax error at or near "%"
LINE 4:                WHERE song_id = %s AND artist_id = %s AND dur...
                                       ^



ProgrammingError: no results to fetch

In [41]:
conncurclose()